In [1]:
%matplotlib inline
from util.cer_util import CER
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import datetime
plt.style.use(r'./stylesheets/eloss.mplstyle')

Need to define <code>rec_key</code> to be for the reconstruction with the same parameters as those passed into the CER constructor here. This is a problem that needs refactoring in the future. Even with this, check that the energies of the reconstructed muons is equal to the energies of muons in <code>cer.muons</code>. (Due to updates to the selection cuts, this fails for the base reconstructions).

In [2]:
rec_key = 'narrow'
cer = CER(e_lims=(1,10))
cer.load_muons()

Loading Data...
Loaded!
Slimming...
Will remove 6997 particles


In [3]:
# Test that the loaded muons are identical to the reconstructed muons
print((cer.muons.backtracked_e - cer.rest_e).to_list() == pd.read_csv(cer.reconstructions[rec_key]).truth.to_list)

False


In [4]:
print(np.abs(pd.read_csv(cer.reconstructions[rec_key]).truth - (cer.muons.backtracked_e - cer.rest_e)).max())

4.224853533685291e-07


For some reason, there is a slight deviation in the true energy tracked by <code>cer.muons</code> and the true energy written to the csv in the narrow energy case. Don't know why this is, but the difference is negligible anyway.

In [5]:
def savefig(fig, name):
    date = datetime.datetime.now()
    month_day = f'{date.month}-{date.day}/'
    path = r'./plots/' + month_day + name
    fig.savefig(path, bbox_inches='tight')
    print(f'Saved to {path}')

In [6]:
def plot_cum_reconstruction(rec_key, mu_idx):
    fit_data = pd.read_csv(cer.fits[rec_key])
    dedxs = cer.generate_eloss(mu_idx)[1]
    loglike, dedxs = cer.cum_binned_like(dedxs, rec_key)
    cum_maxs = np.argmax(loglike, axis=0)
    
    true_e = cer.muons.backtracked_e[mu_idx] - cer.rest_e
    correct_bin = np.digitize(true_e, [0.1] + fit_data['e_max'].to_list()) - 1
    rec_bin = cum_maxs[-1]

    fig = plt.figure()
    lax = fig.gca()
    line, = lax.plot(cum_maxs, ls='-', c='b')
    lax.set_ylabel('reconstruction bin', color='b')
    lax.set_xlabel('datapoint index along track')
    lax.grid(visible=True, axis='y', color='b', alpha=0.2)

    rax = lax.twinx()
    scat, = rax.plot(dedxs, marker='.', c='r')
    rax.set_ylabel('dedx (MeV/cm)', color='r')
    rax.grid(visible=True, axis='y', color='r', linestyle=':', alpha=0.3)
    
    mu_status = 'Bad'
    if rec_bin==correct_bin:
        mu_status = 'Good'
    
    lax.legend([line, scat], 
               [f"Cumulative Reconstruction", f'dE/dx data'],
               loc='upper center', bbox_to_anchor=(0.5, 1.27), shadow=True, framealpha=1, 
               title=f'{mu_status} Muon (index {mu_idx}), {rec_key} reconstruction\nTrue Muon Energy = {true_e:.2f} GeV (bin {correct_bin})\nReconstructed = ({fit_data.e_min[rec_bin]:.2f}, {fit_data.e_max[rec_bin]:.2f}) GeV (bin {rec_bin})')
    return fig

In [7]:
rec_data = pd.read_csv(cer.reconstructions[rec_key])
correctly_reconstructed = ((rec_data.reconstructed_min < rec_data.truth) & (rec_data.truth < rec_data.reconstructed_max))
good_mu_idxs = rec_data[correctly_reconstructed].index
bad_mu_idxs = rec_data[~correctly_reconstructed].index

In [1]:
%%script false --no-raise-error
rng = np.random.default_rng()
chosen_good_idxs = rng.choice(good_mu_idxs, size=50, replace=False)
chosen_bad_idxs = rng.choice(bad_mu_idxs, size=50, replace=False)

with open('cumulative_likelihoods.dat', 'wb') as file:
    for idx in chosen_good_idxs:
        fig = plot_cum_reconstruction(rec_key, idx)
        savefig(fig, f'good_{rec_key}_cumulative_reconstruction_muon_{idx}.png')
        pickle.dump(fig, file)
        plt.close(fig)
        
    for idx in chosen_bad_idxs:
        fig = plot_cum_reconstruction(rec_key, idx)
        savefig(fig, f'bad_{rec_key}_cumulative_reconstruction_muon_{idx}.png')
        pickle.dump(fig, file)
        plt.close(fig)